# Predicting the stock market
In this project, I'll be working with data from the S&P500 Index.

I'll be using historical data on the price of the S&P500 Index to make predictions about future prices. Predicting whether an index goes up or down helps forecast how the stock market as a whole performs. Since stocks tend to correlate with how well the economy as a whole is performs, it can also help with economic forecasts.

The columns of the dataset are:
- Date -- The date of the record.
- Open -- The opening price of the day (when trading starts).
- High -- The highest trade price during the day.
- Low -- The lowest trade price during the day.
- Close -- The closing price for the day (when trading is finished).
- Volume -- The number of shares traded.
- Adj Close -- The daily closing price, adjusted retroactively to include any corporate actions.

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from datetime import datetime

In [2]:
df = pd.read_csv('sphist.csv')
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values('Date')
df

,Date,Open,High,Low,Close,Volume,Adj Close
16589,1950-01-03,16.660000,16.660000,16.660000,16.660000,1.260000e+06,16.660000
16588,1950-01-04,16.850000,16.850000,16.850000,16.850000,1.890000e+06,16.850000
16587,1950-01-05,16.930000,16.930000,16.930000,16.930000,2.550000e+06,16.930000
16586,1950-01-06,16.980000,16.980000,16.980000,16.980000,2.010000e+06,16.980000
16585,1950-01-09,17.080000,17.080000,17.080000,17.080000,2.520000e+06,17.080000
...,...,...,...,...,...,...,...
4,2015-12-01,2082.929932,2103.370117,2082.929932,2102.629883,3.712120e+09,2102.629883
3,2015-12-02,2101.709961,2104.270020,2077.110107,2079.510010,3.950640e+09,2079.510010
2,2015-12-03,2080.709961,2085.000000,2042.349976,2049.620117,4.306490e+09,2049.620117
1,2015-12-04,2051.239990,2093.840088,2051.239990,2091.689941,4.214910e+09,2091.689941


I am going to add three indicators to the data set:
- The average price from the past 5 days.
- The average price for the past 30 days.
- The average price for the past 365 days.

In [3]:
def add_mean_indicator_col(df, num_days, col):
    indicator_name = 'mean_' + col + '_' + str(num_days)
    close_prices = pd.Series(np.array(df[col]), index = np.array(df['Date']))
    means = close_prices.rolling(window = num_days).apply(np.mean)
    means = means.shift()
    
    means = means.reset_index()
    means = means.rename(columns = {'index': 'Date', 0: indicator_name})
    df_new = df.merge(means, left_on = 'Date', right_on = 'Date')
    return df_new

df = add_mean_indicator_col(df, 5, 'Close')
df = add_mean_indicator_col(df, 30, 'Close')
df = add_mean_indicator_col(df, 365, 'Close')

print('head:\n', df.head(35))
print('tail:\n', df.tail(5))

head:
          Date       Open       High        Low      Close     Volume  \
0  1950-01-03  16.660000  16.660000  16.660000  16.660000  1260000.0   
1  1950-01-04  16.850000  16.850000  16.850000  16.850000  1890000.0   
2  1950-01-05  16.930000  16.930000  16.930000  16.930000  2550000.0   
3  1950-01-06  16.980000  16.980000  16.980000  16.980000  2010000.0   
4  1950-01-09  17.080000  17.080000  17.080000  17.080000  2520000.0   
5  1950-01-10  17.030001  17.030001  17.030001  17.030001  2160000.0   
6  1950-01-11  17.090000  17.090000  17.090000  17.090000  2630000.0   
7  1950-01-12  16.760000  16.760000  16.760000  16.760000  2970000.0   
8  1950-01-13  16.670000  16.670000  16.670000  16.670000  3330000.0   
9  1950-01-16  16.719999  16.719999  16.719999  16.719999  1460000.0   
10 1950-01-17  16.860001  16.860001  16.860001  16.860001  1790000.0   
11 1950-01-18  16.850000  16.850000  16.850000  16.850000  1570000.0   
12 1950-01-19  16.870001  16.870001  16.870001  16.870001

In [4]:
df_clean = df.dropna(axis = 0)
df_clean.head()

,Date,Open,High,Low,Close,Volume,Adj Close,mean_Close_5,mean_Close_30,mean_Close_365
365,1951-06-19,22.020000,22.020000,22.020000,22.020000,1100000.0,22.020000,21.800,21.703333,19.447726
366,1951-06-20,21.910000,21.910000,21.910000,21.910000,1120000.0,21.910000,21.900,21.683000,19.462411
367,1951-06-21,21.780001,21.780001,21.780001,21.780001,1100000.0,21.780001,21.972,21.659667,19.476274
368,1951-06-22,21.549999,21.549999,21.549999,21.549999,1340000.0,21.549999,21.960,21.631000,19.489562
369,1951-06-25,21.290001,21.290001,21.290001,21.290001,2440000.0,21.290001,21.862,21.599000,19.502082


In [5]:
df_clean.shape

(16225, 10)

## Train and Test

In [6]:
def train_test(df, features):
    train = df[df['Date'] < datetime(year = 2013, month = 1, day = 1)]
    test = df[df['Date'] >= datetime(year = 2013, month = 1, day = 1)]
    target = 'Close'
    
    lr = LinearRegression()
    lr.fit(train[features], train[target])
    predictions = lr.predict(test[features])
    
    mse = mean_squared_error(predictions, test[target])
    rmse = np.sqrt(mse)
    return rmse

In [7]:
features = list(df_clean.columns)[-3:]
rmse = train_test(df_clean, features)
rmse

22.22006532421986

In [8]:
df = add_mean_indicator_col(df, 5, 'Volume')
df = add_mean_indicator_col(df, 30, 'Volume')
df = add_mean_indicator_col(df, 365, 'Volume')
df_clean = df.dropna(axis = 0)
features = list(df_clean.columns)[-6:]
rmse = train_test(df_clean, features)
rmse

22.23374935410802

Using the average values of the past days of the Volume column does not show improvement in prediction. Let's add the following indicators and see the result:
- The ratio between the average price for the past 5 days, and the average price for the past 365 days.
- The ratio between the standard deviation for the past 5 days, and the standard deviation for the past 365 days.

In [9]:
df['ratio_mean_close'] = df['mean_Close_5'] / df['mean_Close_365']

def add_std_indicator_col(df, num_days, col):
    indicator_name = 'std_' + col + '_' + str(num_days)
    close_prices = pd.Series(np.array(df[col]), index = np.array(df['Date']))
    stds = close_prices.rolling(window = num_days).apply(np.std)
    stds = stds.shift()
    
    stds = stds.reset_index()
    stds = stds.rename(columns = {'index': 'Date', 0: indicator_name})
    df_new = df.merge(stds, left_on = 'Date', right_on = 'Date')
    return df_new

df = add_std_indicator_col(df, 5, 'Close')
df = add_std_indicator_col(df, 365, 'Close')
df['ratio_std_close'] = df['std_Close_5'] / df['std_Close_365']
df_clean = df.dropna(axis = 0)

In [10]:
df_clean.corr()['Close']

Open                0.999900
High                0.999953
Low                 0.999956
Close               1.000000
Volume              0.772817
Adj Close           1.000000
mean_Close_5        0.999793
mean_Close_30       0.999189
mean_Close_365      0.988870
mean_Volume_5       0.780896
mean_Volume_30      0.786908
mean_Volume_365     0.784878
ratio_mean_close    0.047782
std_Close_5         0.722414
std_Close_365       0.816103
ratio_std_close     0.087018
Name: Close, dtype: float64

The ratios do not show a significant effect in reducing error.

Let's check the Date column by creating the following indicators:
- The year component of the date.
- The month component of the date.
- The day component of the date.

In [11]:
df['year'] = df['Date'].dt.strftime('%Y').astype(float)
df['month'] = df['Date'].dt.strftime('%m').astype(float)
df['day'] = df['Date'].dt.strftime('%d').astype(float)

df_clean = df.dropna(axis = 0)
corr_ = df_clean.corr()['Close'].sort_values(ascending = False)
corr_

Close               1.000000
Adj Close           1.000000
Low                 0.999956
High                0.999953
Open                0.999900
mean_Close_5        0.999793
mean_Close_30       0.999189
mean_Close_365      0.988870
year                0.872100
std_Close_365       0.816103
mean_Volume_30      0.786908
mean_Volume_365     0.784878
mean_Volume_5       0.780896
Volume              0.772817
std_Close_5         0.722414
ratio_std_close     0.087018
ratio_mean_close    0.047782
month               0.005684
day                -0.001525
Name: Close, dtype: float64

In [12]:
features =  list(corr_[(corr_ > 0.4) & (corr_ <= 0.9998)].index)
rmse = train_test(df_clean, features)
rmse

21.493341040794412

## Make predictions only one day ahead
For example, train a model using data from 1951-01-03 to 2013-01-02, make predictions for 2013-01-03, and then train another model using data from 1951-01-03 to 2013-01-03, make predictions for 2013-01-04, and so on. This more closely simulates what we'd do if we were trading using the algorithm.

In [13]:
def train_test(df, features, row):
    train = df[df['Date'] < row['Date']]
    test = df[df['Date'] == row['Date']]
    
    if len(train) == 0:
        return np.nan
    else:
        lr = LinearRegression()
        lr.fit(train[features], train['Close'])
        predictions = lr.predict(test[features])
        
        mse = mean_squared_error(predictions, test['Close'])
        rmse = np.sqrt(mse)
        return rmse

In [14]:
rmses = df_clean.apply(lambda row: train_test(df_clean, features, row), axis = 1)
rmse = np.mean(rmses)
rmse

5.418363017460899